In [1]:
import torch
from torchviz import make_dot
from transformers import VisionEncoderDecoderModel, TrOCRProcessor
from PIL import Image
import requests

In [2]:
# Load image
url = 'https://www.visualwatermark.com/images/add-text-to-photos/add-text-to-image-3.webp'
image = Image.open(requests.get(url, stream=True).raw).convert("RGB")

In [3]:
# Load processor and model
processor = TrOCRProcessor.from_pretrained('microsoft/trocr-small-printed')
model = VisionEncoderDecoderModel.from_pretrained('microsoft/trocr-small-printed')

# Prepare input
pixel_values = processor(images=image, return_tensors="pt").pixel_values

# Create a dummy input tensor
dummy_input = torch.randn(1, 3, 224, 224)

# Run the model to get outputs
outputs = model.encoder(pixel_values)

Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-small-printed and are newly initialized: ['encoder.pooler.dense.bias', 'encoder.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
# Generate the visualization
dot = make_dot(outputs.last_hidden_state, params=dict(model.named_parameters()))
dot.format = 'png'
dot.render('model_architecture')

'model_architecture.png'

In [2]:
from graphviz import Digraph

# Tạo đối tượng đồ thị
dot = Digraph(comment='VisionEncoderDecoderModel')

# Encoder
dot.node('E', 'Encoder')
dot.node('E1', 'DeiTModel')
dot.node('E2', 'DeiTEmbeddings')
dot.node('E3', 'DeiTPatchEmbeddings')
dot.node('E4', 'Conv2d')
dot.node('E5', 'DeiTEncoder')
dot.node('E6', 'DeiTLayer x12')
dot.node('E7', 'DeiTAttention')
dot.node('E8', 'DeiTSelfAttention')
dot.node('E9', 'Linear (query/key/value)')
dot.node('E10', 'DeiTSelfOutput')
dot.node('E11', 'Linear (dense)')
dot.node('E12', 'DeiTIntermediate')
dot.node('E13', 'GELUActivation')
dot.node('E14', 'DeiTOutput')
dot.node('E15', 'LayerNorm')
dot.node('E16', 'DeiTPooler')
dot.node('E17', 'Linear')
dot.node('E18', 'Tanh')

# Kết nối các node của encoder
encoder_edges = [
    ('E', 'E1'), ('E1', 'E2'), ('E2', 'E3'), ('E3', 'E4'), ('E4', 'E5'), ('E5', 'E6'),
    ('E6', 'E7'), ('E7', 'E8'), ('E8', 'E9'), ('E7', 'E10'), ('E10', 'E11'), ('E5', 'E12'),
    ('E12', 'E13'), ('E6', 'E14'), ('E14', 'E11'), ('E6', 'E15'), ('E6', 'E16'), ('E16', 'E17'),
    ('E17', 'E18')
]
dot.edges(encoder_edges)

# Decoder
dot.node('D', 'Decoder')
dot.node('D1', 'TrOCRForCausalLM')
dot.node('D2', 'TrOCRDecoderWrapper')
dot.node('D3', 'TrOCRDecoder')
dot.node('D4', 'TrOCRScaledWordEmbedding')
dot.node('D5', 'TrOCRLearnedPositionalEmbedding')
dot.node('D6', 'LayerNorm')
dot.node('D7', 'TrOCRDecoderLayer x6')
dot.node('D8', 'TrOCRAttention')
dot.node('D9', 'Linear (k/q/v_proj)')
dot.node('D10', 'TrOCRAttention (encoder)')
dot.node('D11', 'Linear (k/q_proj)')
dot.node('D12', 'Linear (v_proj)')
dot.node('D13', 'ReLU')
dot.node('D14', 'Linear (fc1)')
dot.node('D15', 'Linear (fc2)')
dot.node('D16', 'Linear (output)')

# Kết nối các node của decoder
decoder_edges = [
    ('D', 'D1'), ('D1', 'D2'), ('D2', 'D3'), ('D3', 'D4'), ('D3', 'D5'), ('D3', 'D6'),
    ('D3', 'D7'), ('D7', 'D8'), ('D8', 'D9'), ('D8', 'D10'), ('D10', 'D11'), ('D10', 'D12'),
    ('D10', 'D13'), ('D7', 'D14'), ('D14', 'D15'), ('D7', 'D16')
]
dot.edges(decoder_edges)

# Kết nối giữa encoder và decoder
dot.edge('E', 'D')

# Lưu và hiển thị đồ thị
dot.render('vision_encoder_decoder_model', format='png')

'vision_encoder_decoder_model.png'